# Naive RAG 간단구성

- 관련 패키지 임포트
- 활용 LLM api정보 설정
- RAG 파이프라인 글로벌 세팅 설정
- 기준 데이터셋 로드
- 기준 데이터셋 전처리
- 벡터스토어인덱스 설정
- 쿼리 엔진 설정

In [ ]:
# 관련 패키지 임포트
import os
from datasets import load_dataset
from llama_index.core import Document
from llama_index.core import VectorStoreIndex
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# 활용 LLM API 정보 설정
os.environ["OPENAI_API_KEY"] = 

In [ ]:
# RAG 파이프라인 글로벌 설정
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)

Settings.llm=OpenAI(model='gpt-3.5-turbo',temperature=0)

In [ ]:
# 기준 데이터셋 로드 - 
ds = load_dataset("lcw99/wikipedia-korean-20240501", split='train[-10:]')

In [ ]:
ds


In [ ]:
# 기준 데이터셋 전처리
data = ds.to_pandas()[['title', 'text']]
data.drop_duplicates(subset='text', keep='first', inplace=True)
data.head()

In [ ]:
# 데이터셋 라마인덱스 다큐먼트 오브젝트로 변환
docs = []

for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        extra_info={'title': row['title']}
    ))

In [ ]:
docs

In [ ]:
# 벡터스토어 인덱스 설정
vector_index = VectorStoreIndex.from_documents(
    docs,
    use_asnyc=True
    )

In [ ]:
# 쿼리 엔진 설정
vector_query_engine = vector_index.as_query_engine(similarity_top_k=2)

- 구축된 Naive-RAG 파이프라인 이용한 기본적인 Q&A 테스팅

In [ ]:
response = vector_query_engine.query('칼 스벤손이 누구야?')

In [ ]:
# 검색에 대한 답변
response.response

In [ ]:
# 소스노드 구조
response.source_nodes

In [ ]:
print(response.source_nodes[0].score)
print(response.source_nodes[1].score)

In [ ]:
# 참조 청크의 사이즈(글자수)
print(response.source_nodes[0].node.start_char_idx)
print(response.source_nodes[0].node.end_char_idx)
print(response.source_nodes[0].node.end_char_idx-response.source_nodes[0].node.start_char_idx)

In [ ]:
# 다른 질문
response = vector_query_engine.query('칼 스벤손이 언제 태어났고, 어떤 월드컵에 참가했지?')

In [ ]:
response.response

In [ ]:
# 소스노드 구조
response.source_nodes

In [ ]:
docs

In [ ]:
# 다른 질문
response = vector_query_engine.query('신창균과 칼 스벤손에 대해서 알려줘')

In [ ]:
response.response